In [1]:
from ultralytics import YOLO
from PIL import Image, ImageDraw
from roboflow import Roboflow
import json
import numpy as np
import cv2
import matplotlib.pyplot as plt
import math
import tensorflow as tf
import cv2

In [2]:
model = YOLO('./runs/segment/train6/weights/best.pt')

In [5]:
def average_pixels_in_block(masks, image_size):
    tresholds = 8
    treshold_range = image_size[0] / tresholds
    average_x_values = [image_size[1] / 2, image_size[1] / 2, image_size[1] / 2, image_size[1] / 2, image_size[1] / 2,  image_size[1] / 2]
    final_points = []
        
    for mask in masks:
        mask = mask[np.argsort(mask[:, 1])]
        index = 0
        current_treshold_range = treshold_range
        
        for i in range(tresholds - 2):
            sum = 0
            left_points = 0
            total_points = 0
            while mask[index][1] <= current_treshold_range:
                sum += mask[index][0]
                total_points += 1
                if mask[index][0] <= image_size[1] / 2:
                    left_points += 1
                index+=1
                if index == mask.shape[0]:
                    index = 0
                    break
            
            if total_points != 0:             
                average_x_values[i] = (sum / int(total_points))
            
            current_treshold_range += treshold_range
            left_points = 0
        index = 0
         
        for i in range(0, tresholds - 2, 2):
            final_points.append((average_x_values[i] + average_x_values[i+1]) / 2)
         
    return final_points

In [4]:
def get_lr_weights(points, image_size):    
    image_size_x = image_size[1]
    weights = []
    for point in points:
        weights.append(point / image_size_x)
    
    return weights

In [9]:
cap = cv2.VideoCapture("videoplayback - Trim.mp4")

while True:
    ret, frame = cap.read() 
    result = model(frame)
    turning_weight = 0
    if len(result) != 0:
        for r in result:
            print(r.names)
            print(r.boxes.cls)
            print(r.orig_shape)
            
            # get the classes and setup empty mask array
            classes = r.boxes.cls.cpu().numpy()
            masks = []
            
            # extract the needed masks
            index = 0
            for mask in r.masks.xy:
                if classes[index] == 5:
                    masks.append(mask)
                index+=1    
            
            if len(masks) == 0:
                continue
            points = average_pixels_in_block(masks, r.orig_shape)
            weights = get_lr_weights(points, r.orig_shape)
            turning_weight = (weights[0] * 0.1 + weights[1] * 0.4 + weights[2] * 0.5) / 1
        

    font = cv2.FONT_HERSHEY_SIMPLEX 
    cv2.putText(frame,  
                f"{turning_weight}",  
                (50, 50),  
                font, 1,  
                (0, 255, 255),  
                2,  
                cv2.LINE_4) 
  
    # Display the resulting frame 
    cv2.imshow('video', frame) 
  
    # creating 'q' as the quit  
    # button for the video 
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break
    
# release the cap object 
cap.release() 
# close all windows 
cv2.destroyAllWindows() 


0: 384x640 2 Curbs, 1 Ground, 1 Sidewalk, 66.1ms
Speed: 4.0ms preprocess, 66.1ms inference, 18.1ms postprocess per image at shape (1, 3, 384, 640)



{0: 'Curb', 1: 'Floor', 2: 'Ground', 3: 'Hole', 4: 'Road', 5: 'Sidewalk', 6: 'Stairs'}
tensor([5., 0., 0., 2.], device='cuda:0')
(1080, 1920)


0: 384x640 2 Curbs, 1 Ground, 1 Sidewalk, 167.9ms
Speed: 3.5ms preprocess, 167.9ms inference, 3.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Curbs, 1 Ground, 1 Sidewalk, 6.2ms
Speed: 2.0ms preprocess, 6.2ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Curbs, 1 Ground, 1 Sidewalk, 6.3ms
Speed: 2.6ms preprocess, 6.3ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Curbs, 1 Ground, 1 Sidewalk, 7.4ms
Speed: 3.2ms preprocess, 7.4ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Curbs, 1 Ground, 1 Sidewalk, 5.3ms
Speed: 3.1ms preprocess, 5.3ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Curbs, 1 Sidewalk, 7.6ms
Speed: 2.0ms preprocess, 7.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)



{0: 'Curb', 1: 'Floor', 2: 'Ground', 3: 'Hole', 4: 'Road', 5: 'Sidewalk', 6: 'Stairs'}
tensor([5., 0., 0., 2.], device='cuda:0')
(1080, 1920)
{0: 'Curb', 1: 'Floor', 2: 'Ground', 3: 'Hole', 4: 'Road', 5: 'Sidewalk', 6: 'Stairs'}
tensor([5., 0., 0., 2.], device='cuda:0')
(1080, 1920)
{0: 'Curb', 1: 'Floor', 2: 'Ground', 3: 'Hole', 4: 'Road', 5: 'Sidewalk', 6: 'Stairs'}
tensor([5., 0., 0., 2.], device='cuda:0')
(1080, 1920)
{0: 'Curb', 1: 'Floor', 2: 'Ground', 3: 'Hole', 4: 'Road', 5: 'Sidewalk', 6: 'Stairs'}
tensor([5., 0., 0., 2.], device='cuda:0')
(1080, 1920)
{0: 'Curb', 1: 'Floor', 2: 'Ground', 3: 'Hole', 4: 'Road', 5: 'Sidewalk', 6: 'Stairs'}
tensor([5., 0., 0., 2.], device='cuda:0')
(1080, 1920)
{0: 'Curb', 1: 'Floor', 2: 'Ground', 3: 'Hole', 4: 'Road', 5: 'Sidewalk', 6: 'Stairs'}
tensor([5., 0., 0.], device='cuda:0')
(1080, 1920)


0: 384x640 2 Curbs, 1 Ground, 1 Sidewalk, 5.1ms
Speed: 2.0ms preprocess, 5.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Curbs, 1 Ground, 1 Sidewalk, 9.0ms
Speed: 2.0ms preprocess, 9.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Curbs, 1 Sidewalk, 7.0ms
Speed: 3.0ms preprocess, 7.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Curbs, 1 Sidewalk, 6.0ms
Speed: 3.0ms preprocess, 6.0ms inference, 2.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Curbs, 1 Sidewalk, 7.1ms
Speed: 1.9ms preprocess, 7.1ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Curbs, 1 Sidewalk, 5.0ms
Speed: 2.0ms preprocess, 5.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)



...

Speed: 3.0ms preprocess, 9.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Curb, 1 Ground, 9.0ms
Speed: 2.0ms preprocess, 9.0ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Ground, 10.0ms
Speed: 2.0ms preprocess, 10.0ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Ground, 9.0ms
Speed: 2.0ms preprocess, 9.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Curbs, 1 Ground, 9.0ms
Speed: 1.0ms preprocess, 9.0ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Ground, 6.8ms
Speed: 1.8ms preprocess, 6.8ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
WARNING  'source' is missing. Using 'source=C:\Users\damem\AppData\Local\Programs\Python\Python310\Lib\site-packages\ultralytics\assets'.

image 1/2 C:\Users\damem\AppData\Local\Programs\Python\Python310\Lib\site-packages\ultralytics\assets\bus.jpg: 640x480 1 Road,

{0: 'Curb', 1: 'Floor', 2: 'Ground', 3: 'Hole', 4: 'Road', 5: 'Sidewalk', 6: 'Stairs'}
tensor([2., 0.], device='cuda:0')
(1080, 1920)
{0: 'Curb', 1: 'Floor', 2: 'Ground', 3: 'Hole', 4: 'Road', 5: 'Sidewalk', 6: 'Stairs'}
tensor([2., 0.], device='cuda:0')
(1080, 1920)
{0: 'Curb', 1: 'Floor', 2: 'Ground', 3: 'Hole', 4: 'Road', 5: 'Sidewalk', 6: 'Stairs'}
tensor([2.], device='cuda:0')
(1080, 1920)
{0: 'Curb', 1: 'Floor', 2: 'Ground', 3: 'Hole', 4: 'Road', 5: 'Sidewalk', 6: 'Stairs'}
tensor([2.], device='cuda:0')
(1080, 1920)
{0: 'Curb', 1: 'Floor', 2: 'Ground', 3: 'Hole', 4: 'Road', 5: 'Sidewalk', 6: 'Stairs'}
tensor([2., 0., 0.], device='cuda:0')
(1080, 1920)
{0: 'Curb', 1: 'Floor', 2: 'Ground', 3: 'Hole', 4: 'Road', 5: 'Sidewalk', 6: 'Stairs'}
tensor([2.], device='cuda:0')
(1080, 1920)


image 2/2 C:\Users\damem\AppData\Local\Programs\Python\Python310\Lib\site-packages\ultralytics\assets\zidane.jpg: 384x640 (no detections), 9.0ms
Speed: 1.5ms preprocess, 9.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)


{0: 'Curb', 1: 'Floor', 2: 'Ground', 3: 'Hole', 4: 'Road', 5: 'Sidewalk', 6: 'Stairs'}
tensor([5., 4.], device='cuda:0')
(1080, 810)
{0: 'Curb', 1: 'Floor', 2: 'Ground', 3: 'Hole', 4: 'Road', 5: 'Sidewalk', 6: 'Stairs'}
tensor([], device='cuda:0')
(720, 1280)


AttributeError: 'NoneType' object has no attribute 'xy'

In [ ]:
result = model("Screenshot_2.png")
uniqueFromArray = []

for r in result:
    print(r.names)
    print(r.boxes.cls)
    print(r.orig_shape)
    
    # get the classes and setup empty mask array
    classes = r.boxes.cls.cpu().numpy()
    masks = []
    
    # extract the needed masks
    index = 0
    for mask in r.masks.xy:
        if classes[index] == 5:
            masks.append(mask)
        index+=1    
    
    points = average_pixels_in_block(masks, r.orig_shape)
    
    temp_mask = masks[0]
    black_image = np.zeros(r.orig_shape)
    black_image = Image.fromarray(black_image)
    temp_mask = temp_mask
    for point in temp_mask:
        black_image.putpixel((math.floor(point[0])-1, math.floor(point[1])-1), 255)
    draw = ImageDraw.Draw(black_image)
    #draw.polygon(temp_mask, fill="black",width=12, outline='blue')
    
    treshold = r.orig_shape[0] / 3
    y = treshold / 2
    for p in points:    
        draw.ellipse((p-10, y-10, p+10, y+10), fill='red', outline='green', width=5)
        y += treshold
    
    weights = get_lr_weights(points, r.orig_shape)
    
    print((weights[0] * 0.1 + weights[1] * 0.4 + weights[2] * 0.5) / 1)
    
    black_image.show()